In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


#  Setup stuff: Connectivity

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


#  Betweeness algorithm

In [7]:

from katana.remote import analytics


l_cntr = 20                                                #  Used to generate a unique column name
   #
print("--")


--


In [8]:

#  We use this counter to generate a unique column name below 
#
l_cntr += 1
   #
l_output_propname     = "bt_" + str(l_cntr).zfill(4)


analytics.betweenness_centrality(
   my_graph,
      #
   result_property_name = l_output_propname,
      #
   num_sources          = 100
   )

print("--")


          0/? [?op/s]

--


# Checking that result above


In [9]:

#  Unfiltered node data
#

l_query = """
   MATCH (n)
   RETURN n
    """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample data,
#
#     +----+-----------------+-------------+-----------+------------------+------------------+-------------+----------+
#     |    |   n.internal_id | n.labels    | n.LABEL   | n.airport_code   | n.airport_name   |   n.bt_0021 | n.type   |
#     |----+-----------------+-------------+-----------+------------------+------------------+-------------+----------|
#     |  0 |               0 | ['Airport'] | Airport   | SJC              | San Jose         |           0 | node     |
#     |  1 | 281474976710656 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   |           4 | node     |
#     |  2 | 562949953421312 | ['Airport'] | Airport   | MKE              | Milwaukee        |           0 | node     |
#     |  3 | 562949953421313 | ['Airport'] | Airport   | DEN              | Denver           |           2 | node     |
#     +----+-----------------+-------------+-----------+------------------+------------------+-------------+----------+


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+------------------+-------------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_code   | n.airport_name   |   n.bt_0021 | n.type   |
|----+-----------------+-------------+-----------+------------------+------------------+-------------+----------|
|  0 |               0 | ['Airport'] | Airport   | SJC              | San Jose         |           0 | node     |
|  1 | 281474976710656 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   |           4 | node     |
|  2 | 562949953421312 | ['Airport'] | Airport   | MKE              | Milwaukee        |           0 | node     |
|  3 | 562949953421313 | ['Airport'] | Airport   | DEN              | Denver           |           2 | node     |
+----+-----------------+-------------+-----------+------------------+------------------+-------------+----------+


In [12]:

#  View output
#

l_query = """

   MATCH (n)
   RETURN n.{0}, ID(n), n.airport_name
   ORDER BY n.{0} DESC
   LIMIT 10
    
    """.format(l_output_propname)

l_result = my_graph.query(l_query)
   #    
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample data,
#
#     +----+-------------+-----------------+------------------+
#     |    |   n.bt_0021 |           ID(n) | n.airport_name   |
#     |----+-------------+-----------------+------------------|
#     |  0 |           4 | 281474976710656 | Chicago O-Hare   |
#     |  1 |           2 | 562949953421313 | Denver           |
#     |  2 |           0 |               0 | San Jose         |
#     |  3 |           0 | 562949953421312 | Milwaukee        |
#     +----+-------------+-----------------+------------------+


          0/? [?op/s]

+----+-------------+-----------------+------------------+
|    |   n.bt_0021 |           ID(n) | n.airport_name   |
|----+-------------+-----------------+------------------|
|  0 |           4 | 281474976710656 | Chicago O-Hare   |
|  1 |           2 | 562949953421313 | Denver           |
|  2 |           0 |               0 | San Jose         |
|  3 |           0 | 562949953421312 | Milwaukee        |
+----+-------------+-----------------+------------------+


#  To remove a given property name from all nodes

In [ ]:

#  It is only required that you run this one time.
#
#  You may change the value i nthe text entry field as often as you wish.
#
from ipywidgets import widgets

my_column  = widgets.Text(
    value        = "bt_0020",
    placeholder  = "type here",
    description  = "Column Name: ",
    disabled     = False
)

my_column


In [ ]:

#  We could run the remove_property() directly, but then we couldn't cleanly catch
#  errors. So, run it inside a UDF.
#

def f_drop_column(i_graph, i_property):
   from katana import distributed
   try:
      i_graph.nodes.remove_property(i_property)
      return distributed.single_host(host=0, result=True)
   except LookupError as e:
      return distributed.single_host(host=0, result=False)
   except Exception as e:
      return distributed.single_host(host=0, result=False)
        

print("Deleting column name: %s" % (my_column.value))
   #
l_column = str(my_column.value)


if (my_graph.run(lambda g: f_drop_column(g, l_column))):
   print("Column ( %s ) deleted." % (my_column.value))
else:
   print("Column ( %s ) not found." % (my_column.value))
